# Étape 2: Ingestion des Données Brutes (Bronze)

## Install de dependances

In [1]:
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.7/341.7 kB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.4/32.4 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# notebooks/02_bronze_ingestion.py - CORRIGÉ
from pyspark.sql import functions as F

def create_bronze_tables():
    """Crée les tables bronze depuis les données sources"""
    
    # 1. Taxi Trips (Parquet 2024)
    print("📊 Étape 1/3: Ingestion des trips taxi...")
    trips_df = spark.read.parquet("/home/iceberg/data/raw/nyc_taxi/taxi_trips/*.parquet")
    
    # Vérifier le schéma des trips
    print("Schéma des trips:")
    trips_df.printSchema()
    
    # Ajout des colonnes de partition
    trips_df = trips_df.withColumn("year", F.year("tpep_pickup_datetime")) \
                      .withColumn("month", F.month("tpep_pickup_datetime")) \
                      .withColumn("day", F.dayofmonth("tpep_pickup_datetime"))
    
    # Repartitionner pour l'écriture (moins de partitions pour éviter OOM)
    trips_df = trips_df.repartition(12, "year", "month")  # 12 partitions pour 12 mois
    
    # Création table Iceberg avec partitions
    trips_df.writeTo("iceberg.bronze.taxi_trips_2024") \
            .partitionedBy("year", "month") \
            .createOrReplace()
    
    print(f"✅ Table bronze.taxi_trips_2024 créée: {trips_df.count()} lignes")
    
    # 2. Taxi Zones (Shapefile)
    print("\n🗺️  Étape 2/3: Ingestion des zones taxi...")
    
    # D'abord essayer de lire un CSV si disponible
    try:
        # Vérifier si un fichier CSV existe
        import os
        csv_path = "/home/iceberg/data/raw/nyc_taxi/taxi_zones/taxi_zones.csv"
        
        if os.path.exists(csv_path):
            zones_df = spark.read.option("header", "true").csv(csv_path)
            print("📁 Fichier CSV trouvé, chargement...")
        else:
            # Sinon utiliser le shapefile
            import geopandas as gpd
            import zipfile
            
            print("📁 Fichier CSV non trouvé, utilisation du shapefile...")
            # Extraction du shapefile
            with zipfile.ZipFile("/home/iceberg/data/raw/nyc_taxi/taxi_zones/taxi_zones.zip", 'r') as zip_ref:
                zip_ref.extractall("/tmp/taxi_zones")
            
            # Lecture avec geopandas
            gdf = gpd.read_file("/tmp/taxi_zones/taxi_zones.shp")
            
            # Conversion en DataFrame Spark
            zones_df = spark.createDataFrame(gdf.drop(columns='geometry'))
            
            # Ajout des colonnes WKT pour le géospatial (optionnel)
            zones_df = zones_df.withColumn("geometry_wkt", 
                                          F.lit(gdf.geometry.to_wkt().tolist()))
    except Exception as e:
        print(f"⚠️  Erreur avec les zones: {e}")
        print("Création d'une table zones vide...")
        from pyspark.sql.types import StructType, StructField, IntegerType, StringType
        
        schema = StructType([
            StructField("LocationID", IntegerType(), True),
            StructField("zone", StringType(), True),
            StructField("borough", StringType(), True)
        ])
        zones_df = spark.createDataFrame([], schema)
    
    # Afficher le schéma des zones
    print("Schéma des zones:")
    zones_df.printSchema()
    
    # Repartitionner en 1 pour éviter trop de fichiers
    zones_df = zones_df.repartition(1)
    
    zones_df.writeTo("iceberg.bronze.taxi_zones").createOrReplace()
    print(f"✅ Table bronze.taxi_zones créée: {zones_df.count()} zones")
    
    # 3. Données Météo
    print("\n🌤️  Étape 3/3: Ingestion des données météo...")
    
    weather_df = spark.read.option("header", "true") \
                          .option("inferSchema", "true") \
                          .csv("/home/iceberg/data/raw/nyc_taxi/weather/central_park_weather_2024.csv")
    
    # Afficher le schéma pour voir les colonnes disponibles
    print("Schéma des données météo:")
    weather_df.printSchema()
    print("Colonnes disponibles:", weather_df.columns)
    print("Aperçu des données:")
    weather_df.show(5)
    
    # CORRECTION : La colonne s'appelle probablement 'time' au lieu de 'datetime'
    # Vérifier quelle colonne contient la date
    if 'time' in weather_df.columns:
        # Si la colonne s'appelle 'time', la convertir en date
        weather_df = weather_df.withColumn("date", F.to_date("time"))
        print("✅ Colonne 'time' convertie en 'date'")
    elif 'date' in weather_df.columns:
        # Si une colonne 'date' existe déjà
        weather_df = weather_df.withColumnRenamed("date", "date_temp").withColumn("date", F.to_date("date_temp"))
        print("✅ Colonne 'date' déjà présente")
    else:
        # Essayer d'utiliser la première colonne qui ressemble à une date
        date_cols = [col for col in weather_df.columns if any(x in col.lower() for x in ['date', 'time', 'datetime', 'timestamp'])]
        if date_cols:
            weather_df = weather_df.withColumn("date", F.to_date(date_cols[0]))
            print(f"✅ Colonne '{date_cols[0]}' utilisée comme date")
        else:
            # Si aucune colonne de date n'est trouvée, créer une date par défaut
            weather_df = weather_df.withColumn("date", F.lit("2024-01-01").cast("date"))
            print("⚠️  Aucune colonne de date trouvée, date par défaut utilisée")
    
    # Repartitionner par date pour l'écriture
    weather_df = weather_df.repartition(12, "date")  # 12 partitions pour 12 mois
    
    weather_df.writeTo("iceberg.bronze.weather_2024") \
              .partitionedBy(F.col("date")) \
              .createOrReplace()
    
    print(f"✅ Table bronze.weather_2024 créée: {weather_df.count()} lignes")
    
    print("\n🎉 Ingestion Bronze terminée avec succès!")
    return trips_df, zones_df, weather_df

In [4]:
create_bronze_tables()

📊 Étape 1/3: Ingestion des trips taxi...


Schéma des trips:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



✅ Table bronze.taxi_trips_2024 créée: 41169720 lignes

🗺️  Étape 2/3: Ingestion des zones taxi...
📁 Fichier CSV non trouvé, utilisation du shapefile...
Schéma des zones:
root
 |-- OBJECTID: long (nullable = true)
 |-- Shape_Leng: double (nullable = true)
 |-- Shape_Area: double (nullable = true)
 |-- zone: string (nullable = true)
 |-- LocationID: long (nullable = true)
 |-- borough: string (nullable = true)
 |-- geometry_wkt: array (nullable = false)
 |    |-- element: string (containsNull = false)



26/01/05 00:19:46 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
                                                                                

✅ Table bronze.taxi_zones créée: 263 zones

🌤️  Étape 3/3: Ingestion des données météo...
Schéma des données météo:
root
 |-- time: timestamp (nullable = true)
 |-- temp: double (nullable = true)
 |-- rhum: integer (nullable = true)
 |-- prcp: double (nullable = true)
 |-- wspd: double (nullable = true)
 |-- pres: double (nullable = true)

Colonnes disponibles: ['time', 'temp', 'rhum', 'prcp', 'wspd', 'pres']
Aperçu des données:
+-------------------+----+----+----+----+------+
|               time|temp|rhum|prcp|wspd|  pres|
+-------------------+----+----+----+----+------+
|2024-01-01 00:00:00| 5.0|  55| 0.0| 9.0|1016.0|
|2024-01-01 01:00:00| 5.0|  55| 0.0| 9.0|1016.0|
|2024-01-01 02:00:00| 5.0|  55| 0.0|13.0|1016.0|
|2024-01-01 03:00:00| 4.0|  60| 0.0| 6.0|1016.0|
|2024-01-01 04:00:00| 4.0|  67| 0.0|10.1|1016.0|
+-------------------+----+----+----+----+------+
only showing top 5 rows

✅ Colonne 'time' convertie en 'date'


✅ Table bronze.weather_2024 créée: 8576 lignes

🎉 Ingestion Bronze terminée avec succès!


(DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double, year: int, month: int, day: int],
 DataFrame[OBJECTID: bigint, Shape_Leng: double, Shape_Area: double, zone: string, LocationID: bigint, borough: string, geometry_wkt: array<string>],
 DataFrame[time: timestamp, temp: double, rhum: int, prcp: double, wspd: double, pres: double, date: date])

In [3]:
# Vérification des tables créées
print("📋 Liste des tables dans le namespace bronze:")
spark.sql("SHOW TABLES IN iceberg.bronze").show()

# Pour une table spécifique
print("\n🔍 Description de la table taxi_trips_2024:")
spark.sql("DESCRIBE EXTENDED iceberg.bronze.taxi_trips_2024").show(truncate=False)

# Vérifier les données
print("\n📊 Aperçu des données (5 premières lignes):")
spark.sql("SELECT * FROM iceberg.bronze.taxi_trips_2024 LIMIT 5").show()

# Vérifier les partitions
print("\n🗂️ Partitions de la table taxi_trips_2024:")
spark.sql("SHOW PARTITIONS iceberg.bronze.taxi_trips_2024").show()

📋 Liste des tables dans le namespace bronze:


Py4JJavaError: An error occurred while calling o37.sql.
: org.apache.iceberg.exceptions.NoSuchNamespaceException: Namespace does not exist: iceberg.bronze
	at org.apache.iceberg.rest.ErrorHandlers$NamespaceErrorHandler.accept(ErrorHandlers.java:173)
	at org.apache.iceberg.rest.ErrorHandlers$NamespaceErrorHandler.accept(ErrorHandlers.java:166)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:224)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:308)
	at org.apache.iceberg.rest.BaseHTTPClient.get(BaseHTTPClient.java:77)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:97)
	at org.apache.iceberg.rest.RESTSessionCatalog.listTables(RESTSessionCatalog.java:388)
	at org.apache.iceberg.catalog.BaseSessionCatalog$AsCatalog.listTables(BaseSessionCatalog.java:79)
	at org.apache.iceberg.rest.RESTCatalog.listTables(RESTCatalog.java:92)
	at org.apache.iceberg.CachingCatalog.listTables(CachingCatalog.java:136)
	at org.apache.iceberg.spark.SparkCatalog.listTa